In [1]:
# ORM -> SQLAlchemy
# Adapter ->
!pip install psycopg2

In [4]:
import psycopg2

connection = psycopg2.connect(
    dbname='testdb',
    user='postgres',
    password='1234',
    host='193.203.191.79',
    port='32001'
)

query = "Select * from users"


In [5]:
import pandas as pd

df = pd.read_sql(query,connection)
df.head(1)

C:\Users\PC1\AppData\Local\Temp\ipykernel_31448\2054633523.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query,connection)


,id,name,email,review,rating
0,1,Raymond Howard,erodriguez@example.net,Partner trouble serious sign bar follow. Assum...,6.8


In [6]:
connection.close()

In [7]:
df.shape

(4011, 5)

In [8]:
reviews = df['review'].values
ratings = df['rating'].values

In [11]:
# token
#!pip install tensorflow

In [13]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(reviews, ratings, test_size=0.3, random_state=42)

In [14]:
from tensorflow.keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer(num_words=10000, oov_token='<OOV>') # 10k kelime max., OOV => out-of vocabulary
tokenizer.fit_on_texts(X_train)

In [16]:
X_train_seq = tokenizer.texts_to_sequences(X_train) #Sözlüğe göre çeviri
X_test_seq = tokenizer.texts_to_sequences(X_test) #Sözlüğe göre çeviri

In [24]:
# Padding -> 50 -> Bütün cümleleri 50 kelimeye tamamla ya da kırp. 40 kelime -> sonuna ya da başına 10 tane boşluk ekle.
# 120 kelime => [50] [50] [50] -> 3. arrayin son 30 kelimesi boşluk.
from tensorflow.keras.preprocessing.sequence import pad_sequences

X_train_pad = pad_sequences(X_train_seq, maxlen=50, padding="post")
X_test_pad = pad_sequences(X_test_seq, maxlen=50, padding="post")

#

In [26]:
#X_train_pad[0], X_train_seq[0]

In [27]:
import tensorflow as tf

In [30]:
# Ödev 1 => Aktivasyon fonksiyonu nedir? Farklı activation func. araştıralım (relu, leaky relu)
# Ödev 2 => Backpropagation nedir? Optimizer nedir? Optimizer türleri.

model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=10000, output_dim=64, input_length=50), # Input katmanı-> Kelimeleri alır vektörlere çevirir
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)), # Cümleyi iki directiondan (yön) oku. # LSTM -> Long Short-Term Memory
    tf.keras.layers.Dense(32, activation='relu'), # 32 output activation fonk.
    tf.keras.layers.Dropout(0.4), # Nöronları rastgele verilen oranda %40 kapatır.
    tf.keras.layers.Dense(1, activation='linear') # Çıktı adeti.
])

model.compile(loss="mean_squared_error", metrics=["mae"], optimizer="adam") #Modelin eğitim öncesi konfigürasyonları

In [31]:
 #  Model eğitime hazır.

history = model.fit(X_train_pad, y_train, epochs=3, validation_data=(X_test_pad, y_test), batch_size=2)

Epoch 1/3
1404/1404 ━━━━━━━━━━━━━━━━━━━━ 13s 8ms/step - loss: 11.0141 - mae: 2.6867 - val_loss: 6.6801 - val_mae: 2.2346
Epoch 2/3
1404/1404 ━━━━━━━━━━━━━━━━━━━━ 12s 8ms/step - loss: 8.2870 - mae: 2.3741 - val_loss: 7.4712 - val_mae: 2.3130
Epoch 3/3
1404/1404 ━━━━━━━━━━━━━━━━━━━━ 11s 8ms/step - loss: 7.1148 - mae: 2.2032 - val_loss: 7.7848 - val_mae: 2.3479


In [32]:
model.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_2 (Embedding)              │ (None, 50, 64)              │         640,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional_2 (Bidirectional)      │ (None, 128)                 │          66,048 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 32)                  │           4,128 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 32)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 1)                   │              33 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 2,130,629 (8.13 MB)

 Trainable params: 710,209 (2.71 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 1,420,420 (5.42 MB)